# Command-Line Interface

> CLI commands for nbdev project overview generation and analysis

In [ ]:
#| default_exp cli

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations
import argparse
import sys
from pathlib import Path
from nbdev.config import get_config
from cjm_nbdev_overview.tree import *
from cjm_nbdev_overview.api_docs import *
from cjm_nbdev_overview.dependencies import *
from cjm_nbdev_overview.parsers import *

## Tree Command

In [ ]:
#| export
def tree_cmd(
    args  # TODO: Add type hint and description
): # Command line arguments - TODO: Add type hint
    "Generate tree visualization for nbdev project"
    # Get project path
    path = Path(args.path) if args.path else None
    
    if args.basic:
        # Basic tree without descriptions
        print(generate_tree(path, show_notebooks_only=args.notebooks_only))
    else:
        # Tree with descriptions
        print(generate_tree_with_descriptions(path))
    
    # Show summary if requested
    if args.summary:
        print(f"\n{get_tree_summary(path)}")
    
    # Generate subdirectory tree if specified
    if args.subdir:
        subdir_path = Path(args.subdir)
        if not subdir_path.is_absolute():
            cfg = get_config()
            subdir_path = cfg.nbs_path / subdir_path
        
        if subdir_path.exists() and subdir_path.is_dir():
            print(f"\n## {subdir_path.name} subdirectory:\n")
            print(generate_subdirectory_tree(subdir_path))
        else:
            print(f"Error: Directory '{args.subdir}' not found", file=sys.stderr)
            sys.exit(1)

## API Documentation Command

In [ ]:
#| export
def api_cmd(
    args  # TODO: Add type hint and description
): # Command line arguments - TODO: Add type hint
    "Generate API documentation for nbdev project"
    # Get project path
    path = Path(args.path) if args.path else None
    
    if args.module:
        # Generate documentation for a specific module
        nb_path = Path(args.module)
        if not nb_path.suffix:
            nb_path = nb_path.with_suffix('.ipynb')
        
        if not nb_path.is_absolute():
            cfg = get_config()
            nb_path = cfg.nbs_path / nb_path
        
        if nb_path.exists():
            try:
                module_info = parse_notebook(nb_path)
                print(generate_module_overview(module_info, show_all=args.all))
            except Exception as e:
                print(f"Error parsing {nb_path}: {e}", file=sys.stderr)
                sys.exit(1)
        else:
            print(f"Error: Notebook '{args.module}' not found", file=sys.stderr)
            sys.exit(1)
    else:
        # Generate full project API documentation
        api_docs = generate_project_api_docs(path, show_all=args.all)
        
        if args.output:
            # Write to file
            output_path = Path(args.output)
            output_path.write_text(api_docs)
            print(f"API documentation written to {output_path}")
        else:
            # Print to stdout
            print(api_docs)

## Dependencies Command

In [ ]:
#| export
def deps_cmd(
    args  # TODO: Add type hint and description
): # Command line arguments - TODO: Add type hint
    "Analyze and visualize module dependencies"
    # Get project path
    path = Path(args.path) if args.path else None
    
    # Build dependency graph
    graph = build_dependency_graph(path, project_name=args.project_name)
    
    if args.module:
        # Show dependencies for specific module
        deps = graph.get_module_dependencies(args.module)
        dependents = graph.get_module_dependents(args.module)
        
        print(f"## Dependencies for {args.module}\n")
        
        if deps:
            print(f"{args.module} depends on:")
            for dep in deps:
                imports = ', '.join(dep.imported_names)
                print(f"  - {dep.target}: {imports}")
        else:
            print(f"{args.module} has no dependencies")
        
        print()
        
        if dependents:
            print(f"Modules that depend on {args.module}:")
            for dep in dependents:
                imports = ', '.join(dep.imported_names)
                print(f"  - {dep.source}: {imports}")
        else:
            print(f"No modules depend on {args.module}")
    
    elif args.matrix:
        # Show dependency matrix
        print("## Dependency Matrix\n")
        print(generate_dependency_matrix(graph))
    
    else:
        # Show mermaid diagram
        direction = "LR" if args.horizontal else "TD"
        diagram = generate_mermaid_diagram(graph, direction=direction, 
                                         show_imports=args.show_imports)
        
        if args.output:
            # Write to file
            output_path = Path(args.output)
            output_path.write_text(diagram)
            print(f"Dependency diagram written to {output_path}")
        else:
            # Print to stdout
            print(diagram)

## Overview Command

In [ ]:
#| export
def overview_cmd(
    args  # TODO: Add type hint and description
): # Command line arguments - TODO: Add type hint
    "Generate complete project overview"
    # Get project path
    path = Path(args.path) if args.path else None
    cfg = get_config()
    
    lines = []
    lines.append(f"# {cfg.lib_name} Project Overview\n")
    
    # Project structure
    lines.append("## Project Structure\n")
    lines.append("```")
    lines.append(generate_tree_with_descriptions(path))
    lines.append("```")
    lines.append(f"\n{get_tree_summary(path)}\n")
    
    # Dependencies
    if not args.no_deps:
        lines.append("\n## Module Dependencies\n")
        graph = build_dependency_graph(path)
        lines.append(generate_mermaid_diagram(graph, direction="LR"))
    
    # API documentation
    if not args.no_api:
        lines.append("\n## API Reference\n")
        api_docs = generate_project_api_docs(path, show_all=args.all)
        # Remove the "## API Reference" header from api_docs since we already added it
        api_docs = api_docs.replace("## API Reference\n\n", "")
        lines.append(api_docs)
    
    overview = '\n'.join(lines)
    
    if args.output:
        # Write to file
        output_path = Path(args.output)
        output_path.write_text(overview)
        print(f"Project overview written to {output_path}")
    else:
        # Print to stdout
        print(overview)

## Update Index Command

In [ ]:
#| export
def update_index_cmd(
    args  # TODO: Add type hint and description
): # Command line arguments - TODO: Add type hint
    "Update index.ipynb with module documentation"
    # Get index path
    index_path = Path(args.index) if args.index else None
    
    try:
        update_index_module_docs(index_path)
        
        # Get the actual path that was updated
        if index_path is None:
            cfg = get_config()
            index_path = cfg.nbs_path / "index.ipynb"
        
        print(f"Successfully updated {index_path} with module documentation")
        
        if args.commit:
            # Stage and commit the changes
            import subprocess
            try:
                subprocess.run(['git', 'add', str(index_path)], check=True)
                subprocess.run(['git', 'commit', '-m', 'Update index.ipynb module documentation'], check=True)
                print("Changes committed to git")
            except subprocess.CalledProcessError as e:
                print(f"Git operations failed: {e}", file=sys.stderr)
                sys.exit(1)
                
    except Exception as e:
        print(f"Error updating index: {e}", file=sys.stderr)
        sys.exit(1)

In [ ]:
#| export
def update_comprehensive_cmd(
    args  # TODO: Add type hint and description
): # Command line arguments - TODO: Add type hint
    "Comprehensively update index.ipynb with all sections"
    # Get index path
    index_path = Path(args.index) if args.index else None
    
    try:
        update_index_comprehensive(
            index_path=index_path,
            include_structure=not args.no_structure,
            include_dependencies=not args.no_dependencies,
            include_cli=not args.no_cli,
            include_modules=not args.no_modules
        )
        
        # Get the actual path that was updated
        if index_path is None:
            cfg = get_config()
            index_path = cfg.nbs_path / "index.ipynb"
        
        sections = []
        if not args.no_structure:
            sections.append("project structure")
        if not args.no_dependencies:
            sections.append("module dependencies")
        if not args.no_cli:
            sections.append("CLI reference")
        if not args.no_modules:
            sections.append("module documentation")
        
        sections_str = ", ".join(sections)
        print(f"Successfully updated {index_path} with {sections_str}")
        
        if args.commit:
            # Stage and commit the changes
            import subprocess
            try:
                subprocess.run(['git', 'add', str(index_path)], check=True)
                subprocess.run(['git', 'commit', '-m', 'Comprehensive update of index.ipynb documentation'], check=True)
                print("Changes committed to git")
            except subprocess.CalledProcessError as e:
                print(f"Git operations failed: {e}", file=sys.stderr)
                sys.exit(1)
                
    except Exception as e:
        print(f"Error updating index: {e}", file=sys.stderr)
        sys.exit(1)

## Main CLI Entry Point

In [ ]:
#| export
def main(
): # TODO: Add type hint
    "Main CLI entry point for nbdev-overview"
    parser = argparse.ArgumentParser(
        prog='nbdev-overview',
        description='Generate comprehensive overviews for nbdev projects'
    )
    
    subparsers = parser.add_subparsers(dest='command', help='Available commands')
    
    # Tree command
    tree_parser = subparsers.add_parser('tree', help='Generate directory tree visualization')
    tree_parser.add_argument('--path', help='Project path (defaults to current nbdev project)')
    tree_parser.add_argument('--basic', action='store_true', help='Show basic tree without descriptions')
    tree_parser.add_argument('--notebooks-only', action='store_true', help='Show only notebooks, not directories')
    tree_parser.add_argument('--summary', action='store_true', help='Include summary statistics')
    tree_parser.add_argument('--subdir', help='Generate tree for specific subdirectory')
    tree_parser.set_defaults(func=tree_cmd)
    
    # API command
    api_parser = subparsers.add_parser('api', help='Generate API documentation')
    api_parser.add_argument('--path', help='Project path (defaults to current nbdev project)')
    api_parser.add_argument('--module', help='Generate docs for specific module')
    api_parser.add_argument('--all', action='store_true', help='Include all items (not just exported)')
    api_parser.add_argument('-o', '--output', help='Output file path')
    api_parser.set_defaults(func=api_cmd)
    
    # Dependencies command
    deps_parser = subparsers.add_parser('deps', help='Analyze module dependencies')
    deps_parser.add_argument('--path', help='Project path (defaults to current nbdev project)')
    deps_parser.add_argument('--module', help='Show dependencies for specific module')
    deps_parser.add_argument('--matrix', action='store_true', help='Show dependency matrix')
    deps_parser.add_argument('--horizontal', action='store_true', help='Use horizontal layout for diagram')
    deps_parser.add_argument('--show-imports', action='store_true', help='Show imported names in diagram')
    deps_parser.add_argument('--project-name', help='Override project name for import detection')
    deps_parser.add_argument('-o', '--output', help='Output file path')
    deps_parser.set_defaults(func=deps_cmd)
    
    # Overview command
    overview_parser = subparsers.add_parser('overview', help='Generate complete project overview')
    overview_parser.add_argument('--path', help='Project path (defaults to current nbdev project)')
    overview_parser.add_argument('--no-deps', action='store_true', help='Skip dependency diagram')
    overview_parser.add_argument('--no-api', action='store_true', help='Skip API documentation')
    overview_parser.add_argument('--all', action='store_true', help='Include all items in API docs')
    overview_parser.add_argument('-o', '--output', help='Output file path')
    overview_parser.set_defaults(func=overview_cmd)
    
    # Update index command
    update_parser = subparsers.add_parser('update-index', help='Update index.ipynb with module documentation')
    update_parser.add_argument('--index', help='Path to index.ipynb (defaults to nbs/index.ipynb)')
    update_parser.add_argument('--commit', action='store_true', help='Commit changes to git')
    update_parser.set_defaults(func=update_index_cmd)
    
    # Update comprehensive command
    comprehensive_parser = subparsers.add_parser('update-comprehensive', help='Comprehensive update of index.ipynb with all sections')
    comprehensive_parser.add_argument('--index', help='Path to index.ipynb (defaults to nbs/index.ipynb)')
    comprehensive_parser.add_argument('--no-structure', action='store_true', help='Skip project structure section')
    comprehensive_parser.add_argument('--no-dependencies', action='store_true', help='Skip module dependencies section')
    comprehensive_parser.add_argument('--no-cli', action='store_true', help='Skip CLI reference section')
    comprehensive_parser.add_argument('--no-modules', action='store_true', help='Skip module documentation section')
    comprehensive_parser.add_argument('--commit', action='store_true', help='Commit changes to git')
    comprehensive_parser.set_defaults(func=update_comprehensive_cmd)
    
    # Parse arguments
    args = parser.parse_args()
    
    # Execute command
    if args.command:
        args.func(args)
    else:
        parser.print_help()

## Console Script Registration

For the CLI to work, we need to register it as a console script in the project settings:

In [ ]:
# Add this to settings.ini under [options.entry_points]:
# console_scripts = nbdev-overview=cjm_nbdev_overview.cli:main

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()